In [1]:
import pickle
import itertools
from collections import Counter
import numpy as np
import networkx as nx
from paths_graph import get_reachable_sets, PathsGraph
from indra.databases import hgnc_client
from indra.preassembler.hierarchy_manager import hierarchies
from indra.sources.biogrid import BiogridProcessor

In [2]:
with open('../indra_depmap_service/_cache/nx_bs_fam_dir_graph_db_refresh_20190702.pkl', 'rb') as f:
    %time graph = pickle.load(f)

CPU times: user 43.2 s, sys: 5.05 s, total: 48.3 s
Wall time: 48.5 s


In [3]:
bp = BiogridProcessor()

INFO: [2019-07-13 16:04:55] indra.sources.biogrid - No data file specified, downloading from BioGrid at https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-3.4.158/BIOGRID-ALL-3.4.158.tab2.zip


## Preprocess Graph

In [ ]:
# Add log weights for paths graph sampling
for u, v, d in graph.edges(data=True):
    belief = d['bs']
    if belief == 1.0:
        belief = 1 - 1e-15
    pg_weight = -np.log(1 - belief)
    d['pg_weight'] = pg_weight

In [ ]:
# Get nodes involved in BioGrid Statements
direct_pairs = set()
for stmt in bp.statements:
    members = stmt.members
    if len(members) != 2:
        print("More than 2 members!")
    direct_pairs.add((members[0].name, members[1].name))
    direct_pairs.add((members[1].name, members[0].name))

In [ ]:
# Create graph from filtered node and edge data
direct_nodes = set()
direct_edges = []

def node_ns_id(node, g):
    node_data = g.nodes[node]
    return (node_data['ns'], node_data['id'])

def get_node_children(node, g, entity_hierarchy):
    node_ns, node_id = node_ns_id(node, g)
    if node_ns != 'FPLX':
        return []
    children = []
    for ch_uri in entity_hierarchy.get_children(entity_hierarchy.get_uri(node_ns, node_id)):
        ch_ns, ch_id = entity_hierarchy.ns_id_from_uri(ch_uri)
        if ch_ns == 'HGNC':
            hgnc_name = hgnc_client.get_hgnc_name(ch_id)
            children.append(hgnc_name)
    return children
            
for u, v, d in graph.edges(data=True):
    # Get any children of the nodes (in case a node is a family), and consider all
    # edges among children/genes as an allowable direct edge
    u_nodes = get_node_children(u, graph, hierarchies['entity'])
    v_nodes = get_node_children(v, graph, hierarchies['entity'])
    u_nodes.append(u)
    v_nodes.append(v)
    for u_i, v_i in itertools.product(u_nodes, v_nodes):
        if (u_i, v_i) in direct_pairs:
            direct_edges.append((u, v, d))
            direct_nodes.add(u)
            direct_nodes.add(v)
            break
        
direct_graph = nx.DiGraph()
direct_graph.add_nodes_from([(u, graph.nodes[u]) for u in direct_nodes])
direct_graph.add_edges_from(direct_edges)

## Pathfinding with path filtering to eliminate family cycles

In [ ]:
# Path filtering

def valid_path(path, source_graph, entity_hierarchy, is_pg=False):
    # Assemble list of groundings and node component IDs
    node_groundings = set()
    components = set()
    cycle = False
    for node in path:
        # If this is a node from a paths graph, get the node name
        if is_pg:
            node = node[1]
        # Check for cycles involving groundings (nodes may not be in hierarchy)
        node_ns, node_id = node_ns_id(node, source_graph)
        if (node_ns, node_id) in node_groundings:
            cycle = True
            break
        else:
            node_groundings.add((node_ns, node_id))
        # Also check for cycles involving components
        node_uri = entity_hierarchy.get_uri(node_ns, node_id)
        component_id = entity_hierarchy.components.get(node_uri)
        if component_id is not None and component_id in components:
            cycle = True
            break
        else:
            components.add(component_id)
    return True if not cycle else False

# Shortest paths

def shortest_paths(path_graph, source_graph, source, target, weight_key, num_paths, is_pg=False):
    paths = []
    for i, path in enumerate(nx.shortest_simple_paths(path_graph, source,
                                                      target, weight=weight_key)):
        if valid_path(path, source_graph, hierarchies['entity'], is_pg):
            paths.append(path)
            print(len(paths), path)
        if len(paths) >= num_paths:
            break
    print("Filtered %d total paths to find %d valid ones" % (i+1, num_paths))

## Set source and target

In [ ]:
source = 'SIK3'
target = 'JUN'

## Shortest paths (original graph)

In [ ]:
num_paths = 30
shortest_paths(direct_graph, direct_graph, source, target, 'weight', num_paths)

## Prepare paths graphs

In [ ]:
fwd_rs, back_rs = get_reachable_sets(direct_graph, source, target, max_depth=5,
                                           signed=False)

In [ ]:
length = 3

In [ ]:
pg_samp = PathsGraph.from_graph(direct_graph, source, target, length,
                          fwd_reachset=fwd_rs, back_reachset=back_rs, signed=False,
                          weight_key='pg_weight')
pg_short = PathsGraph.from_graph(direct_graph, source, target, length,
                           fwd_reachset=fwd_rs, back_reachset=back_rs, signed=False,
                           weight_key='weight')

First let's use the paths graph to get the shortest simple paths, and filter out paths involving loops:

In [ ]:
num_paths = 30

# Loop over shortest paths, filtering out paths involving cycles or family cycles
shortest_paths(pg_short.graph, direct_graph, pg_short.source_node, pg_short.target_node, 'weight', num_paths, is_pg=True)

Now we run sampling, using the paths graph with the edge weights normalized accordingly:

In [ ]:
num_samples = 10000
paths = pg_samp.sample_cf_paths(num_samples)

In [ ]:
ctr = Counter(paths)
path_ctr = sorted([(k, v) for k, v in ctr.items()], key=lambda x: x[1], reverse=True)

In [ ]:
path_ctr

In [ ]:
def bel(s, t):
    return graph[s][t]['bs']

In [ ]:
bel('EGFR', 'mTORC2')

In [ ]:
import networkx as nx

In [ ]:
pg.graph[(0, 'EGFR')][(1, 'AFAP1')]

In [ ]:
pg.enumerate_paths()

In [ ]:
pg.count_paths()

In [ ]:
pg.count_cf_paths()